In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, json
import re

In [4]:
RAW_DIR = Path("/Users/monicaekstein/Downloads/ADS505 Pharma Sales Data")

daily_raw = pd.read_csv(RAW_DIR / "salesdaily.csv")
hourly_raw = pd.read_csv(RAW_DIR / "saleshourly.csv")
weekly_raw = pd.read_csv(RAW_DIR / "salesweekly.csv")
monthly_raw = pd.read_csv(RAW_DIR / "salesmonthly.csv")

### Logistic regression

In [23]:
from pathlib import Path
import numpy as np, pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    f1_score, classification_report, confusion_matrix
)
import json, os

SEED = 42
np.random.seed(SEED)

# Setup
try:
    DATA_DIR = Path(RAW_DIR)
except NameError:
    DATA_DIR = Path(".")
ARTIFACTS = DATA_DIR / "artifacts"
ARTIFACTS.mkdir(parents=True, exist_ok=True)

# Load weekly and construct target series
wk = pd.read_csv(DATA_DIR / "salesweekly.csv")
wk.columns = [c.strip().lower().replace(" ", "_") for c in wk.columns]
wk["datum"] = pd.to_datetime(wk["datum"])
value_cols = [c for c in wk.columns if c != "datum"] # all 8 ATC cols
wk["units"] = wk[value_cols].sum(axis=1)

series = (wk.sort_values("datum")
            .set_index("datum")["units"]
            .asfreq("W", fill_value=0))  # regular weekly grid
series.name = "units"

# Feature engineering
def make_features(s: pd.Series) -> pd.DataFrame:
    """Plain lags/rolls + simple calendar"""
    df = pd.DataFrame({"units": s})
    for l in range(1, 13):
        df[f"lag_{l}"] = s.shift(l)
    df["roll_mean_4"]  = s.rolling(4).mean()
    df["roll_mean_12"] = s.rolling(12).mean()
    iso = df.index.isocalendar()
    df["weekofyear"] = iso.week.astype(int)
    df["month"] = df.index.month.astype(int)
    df["year"]  = df.index.year.astype(int)
    df["units_next"] = df["units"].shift(-1)   # label base (no leakage)
    return df.dropna()

model_df = make_features(series).reset_index(names="datum")
model_df = model_df.sort_values("datum").reset_index(drop=True)

# 85/15 chronological split (TRAIN/TEST)
cutoff_date = model_df["datum"].quantile(0.85)  # ~early2014–mid2018 train, late2018–2019 test
model_df["is_test"] = model_df["datum"] >= cutoff_date
train_mask = ~model_df["is_test"]

print(f"Train range: {model_df.loc[train_mask,'datum'].min().date()} → "
      f"{model_df.loc[train_mask,'datum'].max().date()}  "
      f"| Test range: {model_df.loc[~train_mask,'datum'].min().date()} → "
      f"{model_df.loc[~train_mask,'datum'].max().date()}")

# Train-only labeling (leak-free)
def label_surge_leak_free(model_df: pd.DataFrame, train_mask: pd.Series,
                          min_pos_frac=0.02, min_pos_floor=5):
    """
    Define 'surge_next' using TRAIN ONLY information.
    """
    y_next_train = model_df.loc[train_mask, "units_next"]
    min_pos = max(min_pos_floor, int(min_pos_frac * train_mask.sum()))
    surge = pd.Series(0, index=model_df.index, dtype=int)
    rule = None

    # High-quantile on TRAIN
    for q in [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60]:
        thr = float(np.quantile(y_next_train, q))
        tmp = (model_df["units_next"] >= thr).astype(int)
        if tmp[train_mask].sum() >= min_pos:
            surge = tmp; rule = f"q{int(q*100)} (thr={thr:.2f})"; break

    # %-jump vs current week on TRAIN
    if rule is None:
        pct_jump = (model_df["units_next"] - model_df["units"]) / np.maximum(1.0, model_df["units"])
        for theta in [0.30, 0.25, 0.20, 0.15, 0.10]:
            tmp = (pct_jump >= theta).astype(int)
            if tmp[train_mask].sum() >= min_pos:
                surge = tmp; rule = f"pct_jump≥{int(theta*100)}%"; break

    # Top-k in TRAIN
    if rule is None:
        k = min_pos
        idx_top = y_next_train.sort_values(ascending=False).index[:k]
        surge.loc[idx_top] = 1
        thr = float(y_next_train.loc[idx_top].min())
        rule = f"top-{k} by next-week units (min={thr:.2f})"

    return surge, rule

surge, rule = label_surge_leak_free(model_df, train_mask)
model_df["surge_next"] = surge
print(f"Label rule: {rule}")

X_cols = [c for c in model_df.columns
          if c not in {"datum", "is_test", "units", "units_next", "surge_next"}]

# Final TRAIN/TEST frames
train_df = model_df.loc[train_mask].reset_index(drop=True)
test_df  = model_df.loc[~train_mask].reset_index(drop=True)

X_train, y_train = train_df[X_cols], train_df["surge_next"].astype(int)
X_test,  y_test  = test_df[X_cols],  test_df["surge_next"].astype(int)

if y_train.nunique() < 2:
    raise ValueError("Training set still 1-class; relax labeling or widen the train window.")

print(f"Train: {X_train.shape}, pos={y_train.sum()} ({y_train.mean():.1%}) | "
      f"Test: {X_test.shape}, pos={y_test.sum()} ({y_test.mean():.1%})")

# Model + time-aware CV (on TRAIN only)
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logit",  LogisticRegression(
        solver="liblinear", class_weight="balanced",
        max_iter=5000, random_state=SEED
    )),
])

param_grid = {
    "logit__penalty": ["l2", "l1"],
    "logit__C": [0.05, 0.1, 0.25, 0.5, 1.0, 2.0],
}

# Expanding-window folds by week; skip folds with single class
weeks = np.array(sorted(train_df["datum"].unique()))
W = len(weeks)
n_splits = 4
edges = np.linspace(0, W, num=n_splits + 2, dtype=int)

splits = []
for k in range(n_splits):
    tr_weeks = weeks[:edges[k+1]]
    te_weeks = weeks[edges[k+1]:edges[k+2]]
    tr_idx = np.where(train_df["datum"].isin(tr_weeks))[0]
    te_idx = np.where(train_df["datum"].isin(te_weeks))[0]
    if tr_idx.size == 0 or te_idx.size == 0:
        continue
    if np.unique(y_train.iloc[tr_idx]).size < 2 or np.unique(y_train.iloc[te_idx]).size < 2:
        continue
    splits.append((tr_idx, te_idx))

# Backup: internal 80/20 holdout inside TRAIN
if not splits:
    cut = int(0.8 * len(train_df))
    tr_idx = np.arange(cut)
    te_idx = np.arange(cut, len(train_df))
    if np.unique(y_train.iloc[tr_idx]).size == 2 and np.unique(y_train.iloc[te_idx]).size == 2:
        splits = [(tr_idx, te_idx)]
    else:
        raise ValueError("Could not form valid CV folds with both classes.")

print(f"Built {len(splits)} time-aware CV fold(s).")

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="average_precision",   # PR-AUC is the right thing under imbalance
    cv=splits,
    n_jobs=-1,
    refit=True,
    verbose=0,
    error_score=0.0
)
gs.fit(X_train, y_train)
best = gs.best_estimator_
print("Best params:", gs.best_params_, "| CV PR-AUC:", f"{gs.best_score_:.3f}")

# Pick threshold on CV (TRAIN only)
def best_tau_for(y_true, p):
    pr, rc, thr = precision_recall_curve(y_true, p)
    if len(thr) == 0:  # degenerate
        return 0.5
    f1 = 2*pr*rc/(pr+rc+1e-12)
    return float(thr[np.argmax(f1[:-1])])

taus = []
for tr_idx, te_idx in splits:
    # re-fit best params on each fold's train
    fold_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("logit",  LogisticRegression(
            solver="liblinear",
            penalty=best.named_steps["logit"].penalty,
            C=best.named_steps["logit"].C,
            class_weight="balanced",
            max_iter=5000,
            random_state=SEED
        )),
    ])
    fold_pipe.fit(X_train.iloc[tr_idx], y_train.iloc[tr_idx])
    p_te = fold_pipe.predict_proba(X_train.iloc[te_idx])[:,1]
    if len(np.unique(y_train.iloc[te_idx])) == 2:
        taus.append(best_tau_for(y_train.iloc[te_idx], p_te))

tau_cv = float(np.median(taus)) if taus else 0.5
print(f"Chosen decision threshold (CV median): τ={tau_cv:.3f}")

# Probability calibration (fit on TRAIN)
try:
    calibrated = CalibratedClassifierCV(estimator=best, method="isotonic", cv=3)
except TypeError:
    calibrated = CalibratedClassifierCV(base_estimator=best, method="isotonic", cv=3)

try:
    calibrated.fit(X_train, y_train)
    p_test_cal = calibrated.predict_proba(X_test)[:,1]
except Exception:
    try:
        calibrated = CalibratedClassifierCV(estimator=best, method="sigmoid", cv=3)
    except TypeError:
        calibrated = CalibratedClassifierCV(base_estimator=best, method="sigmoid", cv=3)
    calibrated.fit(X_train, y_train)
    p_test_cal = calibrated.predict_proba(X_test)[:,1]

p_test_raw = best.predict_proba(X_test)[:,1]

# Metrics on TEST (robust to single-class)
def summarize(y, p, tau, tag):
    metrics = {"tag": tag}
    has_both = (np.unique(y).size == 2)
    if has_both:
        roc = roc_auc_score(y, p)
        prauc = average_precision_score(y, p)
        yhat = (p >= tau).astype(int)
        f1 = f1_score(y, yhat)
        print(f"[{tag}] ROC-AUC={roc:.3f} | PR-AUC={prauc:.3f} | F1@τ={tau:.3f} ⇒ {f1:.3f}")
        print(classification_report(y, yhat, digits=3))
        print("Confusion matrix:\n", confusion_matrix(y, yhat))
        metrics.update({"roc_auc": float(roc), "pr_auc": float(prauc), "f1": float(f1)})
    else:
        pos = int(y.sum())
        print(f"[{tag}] Test has a single class (positives={pos}). "
              "ROC/PR/F1 undefined; skipping threshold metrics.")
        metrics.update({"roc_auc": None, "pr_auc": None, "f1": None})
    return metrics

m_raw = summarize(y_test, p_test_raw, tau_cv, "uncalibrated")
m_cal = summarize(y_test, p_test_cal, tau_cv, "calibrated")

# Sanity baseline (on TEST)
pct90 = np.nanpercentile(train_df["lag_1"], 90)
yhat_base = (X_test["lag_1"] >= pct90).astype(int)
if np.unique(y_test).size == 2:
    print("[baseline] F1:", f1_score(y_test, yhat_base))
else:
    print("[baseline] skipped (test single-class)")

# Coefficients (odds ratios)
coef = best.named_steps["logit"].coef_.ravel()
odds = pd.Series(np.exp(coef), index=X_train.columns).sort_values(ascending=False)
print("\nTop positive odds-ratio features:\n", odds.head(10))
print("\nTop negative odds-ratio features:\n", odds.tail(10))

# Error analysis + surge watchlist (on TEST)
err = test_df[["datum"]].copy()
err["y"] = y_test.to_numpy()
err["p_cal"] = p_test_cal
err["yhat"] = (err["p_cal"] >= tau_cv).astype(int)
fn = err[(err["y"] == 1) & (err["yhat"] == 0)].sort_values("p_cal").head(5)
fp = err[(err["y"] == 0) & (err["yhat"] == 1)].sort_values("p_cal", ascending=False).head(5)
print("\nTop FN (actual=1, predicted=0):\n", fn)
print("\nTop FP (actual=0, predicted=1):\n", fp)

watchlist = err.sort_values("p_cal", ascending=False).reset_index(drop=True)

# Train-only surge uplift
mu_pos = train_df.loc[train_df["surge_next"] == 1, "units_next"].mean()
mu_neg = train_df.loc[train_df["surge_next"] == 0, "units_next"].mean()
uplift = float(max(0.0, (mu_pos - mu_neg) / max(mu_neg, 1e-6)))
print(f"\nEstimated average uplift on surge weeks (train-only): {uplift:.2%}")

# Save artifacts
meta = {
    "best_params": gs.best_params_,
    "cv_pr_auc": float(gs.best_score_),
    "tau_cv": float(tau_cv),›
    "cutoff_date": str(cutoff_date.date()),
    "test_rows": int(len(test_df)),
    "test_pos": int(y_test.sum()),
    "label_rule": str(rule),
    "uplift_pct": uplift,
    "seed": SEED,
}
(pd.Series(meta, dtype="object")
   .to_json(ARTIFACTS / "logit_meta.json", indent=2))

pd.DataFrame({
    "date": test_df["datum"],
    "y": y_test,
    "p_raw": p_test_raw,
    "p_cal": p_test_cal,
    "yhat": (p_test_cal >= tau_cv).astype(int)
}).to_csv(ARTIFACTS / "logit_test_preds.csv", index=False)

odds.to_csv(ARTIFACTS / "logit_odds_ratio.csv")
watchlist[["datum","y","p_cal","yhat"]].to_csv(ARTIFACTS / "logit_watchlist.csv", index=False)

print(f"\nSaved artifacts → {ARTIFACTS.resolve()}")

Train range: 2014-03-30 → 2018-12-02  | Test range: 2018-12-09 → 2019-10-06
Label rule: q95 (thr=589.84)
Train: (245, 17), pos=13 (5.3%) | Test: (44, 17), pos=4 (9.1%)
Built 4 time-aware CV fold(s).
Best params: {'logit__C': 0.05, 'logit__penalty': 'l1'} | CV PR-AUC: 0.459
Chosen decision threshold (CV median): τ=0.628
[uncalibrated] ROC-AUC=0.888 | PR-AUC=0.392 | F1@τ=0.628 ⇒ 0.462
              precision    recall  f1-score   support

           0      0.971     0.850     0.907        40
           1      0.333     0.750     0.462         4

    accuracy                          0.841        44
   macro avg      0.652     0.800     0.684        44
weighted avg      0.913     0.841     0.866        44

Confusion matrix:
 [[34  6]
 [ 1  3]]
[calibrated] ROC-AUC=0.875 | PR-AUC=0.330 | F1@τ=0.628 ⇒ 0.286
              precision    recall  f1-score   support

           0      0.927     0.950     0.938        40
           1      0.333     0.250     0.286         4

    accuracy          